# Quantum Ansatz Optimisation Algorithm for the Quadratic Assignment Problem

In [31]:
#Initialisation cell
from docplex.mp.model import Model
from qiskit.providers.ibmq.managed import IBMQJobManager

from qiskit import BasicAer, Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver,NumPyEigensolver,QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes #(uses CX entangling) 
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.circuit.library import TwoLocal
#from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance

from docplex.mp.model import Model
from qiskit.optimization import QuadraticProgram
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.applications.ising import docplex
from qiskit.optimization.algorithms import MinimumEigenOptimizer

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import pandas as pd
from   matplotlib import cm
from   matplotlib.ticker import LinearLocator, FormatStrFormatter
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

# importing Qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from qiskit.providers.ibmq      import least_busy
from qiskit.tools.monitor       import job_monitor

The functions below are for inputting the csv data (that is in the QAPLIB format) and making the matrices numpy arrays.

In [12]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

Below is the fitness function for the QAP using the  Quadratic 0–1 formulation

In [13]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length))
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Below, IBMQ account is loaded and the backends displayed

In [4]:
from qiskit import IBMQ

 # Load account from disk

IBMQ.save_account("3cd4df76e664d15c0a0dde90b7ce30dd1c5f0040004c5b46c8f6055623a104b2cc23d8473c156efa1dfc6c12fbb052c1c5b25a7b9f2ad26585ebb1a93f2be810", overwrite = True)
IBMQ.load_account()
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='physics')>]

In [5]:
provider = IBMQ.get_provider(hub='ibm-q-wits', group='internal', project='physics')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_johannesburg') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_rochester') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('

These functions are needed to find and convert the answer

In [14]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n))
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [15]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    feasible binary 1D numpyarray
    probability of this answer
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    
    feasible=False
    while feasible==False and counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        frequency = bestinarray[counter][1]
        counter += 1
        
    if feasible == False:
        return feasible
    else:
        return bestasint[1:], frequency/total

This function outputs the QAOA solutions

In [49]:
def testing_quantum(machines, instances):
    """
    Input:
    machines: list of strings of the names of available machines
    instances: list of instances
    
    Output:
    outmatrix : [time, eigensolution value,machine value,instance]
    """
    instancesolutions = []
    for ins in instances:
        #get matrix
        datamatrix = CSVtoNumpyArray(ins)
        MatrixLoc = datamatrix[0]
        MatrixFlow = datamatrix[1]
        n = len(MatrixLoc)
        
        # Create an instance of a model and variables.
        thename = "qap" + str(n)
        mdl = Model(name=thename)
        x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}
        
        # Object function
        qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
        mdl.minimize(qap_func)

        # Constraints
        for i in range(n):
            mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
        for p in range(n):
            mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
        print(mdl.export_to_string())
        qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error
        
        #Eigensolver
        ee = NumPyEigensolver(qubitOp_docplex, k=1)#potential error
        result1 = ee.run() #potential error
        print('Eigen energy:', result1['eigenvalues'])
        print('Eigen QAP objective:', np.real(result1['eigenvalues'])[0] + offset_docplex)
        
        #QAOA
        machinesolutions = []
        for j in machines:
            seed = 10598
            spsa = SPSA(max_trials=300)
            backend = Aer.get_backend(j)
            problem = QuadraticProgram(qubitOp_docplex)
            #ry = RealAmplitudes(qubitOp_docplex.num_qubits, entanglement='linear')
           
            #backend = provider.get_backend(j)
            quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, skip_qobj_validation = False, shots = 100)
            qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=5, quantum_instance=quantum_instance)
            result = qaoa.run(backend)
            #print(result["eigenstate"])
            print('QAOA energy:', result['eigenvalue'])
            print('QAOA time:', result['optimizer_time'])
            print('QAOA QAP objective:', result['eigenvalue'] + offset_docplex)
            x = choose_best_feasible(result['eigenstate'])
            print(x)
            soln = qap_value(x[0],MatrixFlow,MatrixLoc)
            machinesolutions.append([soln,j,result['optimizer_time'],x])
        instancesolutions.append([ins,result1['eigenvalues']+offset_docplex, machinesolutions])
        print(instancesolutions)
    return instancesolutions
    

In [50]:
#Uncomment to run on the machines

#machines = ['ibmq_qasm_simulator', 'ibmqx2','ibmq_16_melbourne', 'ibmq_vigo', 'ibmq_ourense', 'ibmq_valencia', 
#'ibmq_london', 'ibmq_burlington', 'ibmq_johannesburg', 'ibmq_rochester', 'ibmq_cambridge', 'ibmq_paris']
#instances = ['made4.csv','made4.csv','made5.csv','made6.csv','made7.csv','made8.csv']

#eigens = testing_quantum(machines, instances)

In [51]:
machines = ['qasm_simulator']
instances = ['made4.csv']

testing_quantum(machines, instances)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x_

[['made4.csv',
  array([790.+0.j]),
  [[926,
    'qasm_simulator',
    391.9352185726166,
    (array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]), 0.0009765625)]]]]